In [1]:
import pandas as pd
import numpy as np
import scipy
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re
from scipy.linalg import norm
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [2]:
# Загружаем стоп слова для русского и английского языка
stop1 = list(stopwords.words('english'))
stop2 = list(stopwords.words('russian'))
stop = stop1 + stop2

lemmatizer = WordNetLemmatizer()

# Функция для предобработки. Удаление знаков пуктуации, приведение к нижнему регистру, лемматизация.
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[.,"\'-?:!;]', "", line)
    line = nltk.word_tokenize(line)
    line = ' '.join([lemmatizer.lemmatize(w) for w in line])
    return line

In [3]:
# Читаем наш датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497819 entries, 0 to 497818
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   pair_id       497819 non-null  int64 
 1   name_1        497819 non-null  object
 2   name_2        497819 non-null  object
 3   is_duplicate  497819 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 15.2+ MB


In [6]:
# Посмотрим сколько одинаковых пар, а сколько разных.
df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [7]:
# Разбиваем датафрейм на на части с копиями и без них.

df_0 = df[df['is_duplicate'] == 0]
df_1 = df[df['is_duplicate'] == 1]

name_1_df_0 = list(df_0['name_1'])
name_2_df_0 = list(df_0['name_2'])

name_1_df_1 = list(df_1['name_1'])
name_2_df_1 = list(df_1['name_2'])

In [9]:
# Объединим два столбца, посмотрим сколько всего названий.
text = list(df['name_1']) + list(list(df['name_2']))

print(len(text))

995638


In [10]:
# Оставим только уникальные названия.
text_unic = list(set(text))

print(len(text_unic))

18022


In [11]:
# Посмотрим на результат предобработки
print(preprocessing(text[12]))

bestocean worldwide logistics inc


In [12]:
# Будем делать эмбеддинги с помощью TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing, analyzer = 'word',stop_words=stop)

tfidf_vectorizer.fit(text_unic)

E:\10\PROGRAMMS\anaconda\envs\main\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


TfidfVectorizer(preprocessor=<function preprocessing at 0x000001E4556899D0>,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [13]:
# Пример текста. Предобработаем его, вычислим эмбеддинги, а потом посмотрим на косинусное сходство.
text1 = preprocessing('Carlisle Coatings & Waterproofing, Inc.' )
text2 = preprocessing('Carlisle Coatings & Wtrprfng')

print(text1)
print(text2)

carlisle coating & waterproofing inc
carlisle coating & wtrprfng


In [14]:
# Сделаем эмбеддинги двух примеров названий.
text1_vec = tfidf_vectorizer.transform([text1])
text2_vec = tfidf_vectorizer.transform([text2])

print(text1_vec.shape)
print(text2_vec.shape)

(1, 15932)
(1, 15932)


In [15]:
# Посмотрим на косинусное сходство двух схожих текстов.
print(cosine_similarity(text1_vec, text2_vec))
print(cosine_similarity(text1_vec, text2_vec)[0][0])

[[0.58029538]]
0.5802953765169023


In [16]:
# Посмотрим на косинусное сходство двух несхожих текстов.
print(cosine_similarity(tfidf_vectorizer.transform(['Iko Industries Ltd.']), tfidf_vectorizer.transform(['Enormous Industrial Trade Pvt., Ltd.'])))

[[0.03765532]]


In [1]:
# Вычислим косинусное сходство для всех пар из нашего датасета.


def prep(text):
    return tfidf_vectorizer.transform([preprocessing(text)])


pair_0 = []
pair_1 = []

for i in range(len(name_1_df_0) ):
    
    pair_0.append( cosine_similarity( prep(name_1_df_0[i] ),prep(name_2_df_0[i])  ))

    
for i in range(len(name_1_df_1) ):
    
    pair_1.append( cosine_similarity( prep(name_1_df_1[i] ),prep(name_2_df_1[i])  ))    

In [19]:
# Посмотрим на косинусное сходство первых 50 непохожиш пар.
print(pair_0[:50])

[array([[0.03765532]]), array([[0.19197232]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.34142763]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.12334998]]), array([[0.12495284]]), array([[0.06036677]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.10729315]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.13716193]]), array([[0.15815033]]), array([[0.13965938]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.19253311]]), array([[0.]]), array([[0.10244252]]), array([[0.29215175]]), array([[0.17287329]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.04771968]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.17103228]]), array([[0.]]), array([[0.]]), array([[0.32738178]]), array([[0.]]), array([[0.]]), array([[0.]])]


In [20]:
# Посмотрим на косинусное сходство первых 50 похожиш пар.
print(pair_1[:50])

[array([[0.77822582]]), array([[0.4210751]]), array([[0.55818533]]), array([[0.52995888]]), array([[0.1541044]]), array([[0.25176037]]), array([[0.56044089]]), array([[0.58029538]]), array([[0.35928688]]), array([[0.55887625]]), array([[0.77159317]]), array([[0.48812509]]), array([[0.33527645]]), array([[0.41091764]]), array([[0.48404833]]), array([[0.79613119]]), array([[1.]]), array([[0.24603778]]), array([[0.33458772]]), array([[0.80027732]]), array([[0.57447577]]), array([[0.54399832]]), array([[0.37463725]]), array([[0.48365744]]), array([[0.33466634]]), array([[0.64760312]]), array([[1.]]), array([[0.80039734]]), array([[1.]]), array([[0.37745632]]), array([[0.54938373]]), array([[0.34161408]]), array([[0.6814642]]), array([[0.40513767]]), array([[0.]]), array([[0.47853116]]), array([[0.48828846]]), array([[0.91170319]]), array([[0.55400584]]), array([[0.88026406]]), array([[0.87559488]]), array([[0.16927868]]), array([[0.4057115]]), array([[0.31095996]]), array([[0.39413544]]), 

In [21]:
print(len(pair_0))
print(len(pair_1))

494161
3658


In [24]:
# Выведем долю пар схожих и отличных по косинусному сходству по порогу 0.3

pair_0_trashold_05 = [i for i in pair_0 if i < 0.3]
pair_1_trashold_05 = [i for i in pair_1 if i>= 0.3]

print(len(pair_0_trashold_05), "  ",len(pair_0_trashold_05)/len(pair_0)*100)

print(len(pair_1_trashold_05), "  ",len(pair_1_trashold_05)/len(pair_1)*100)

463144    93.72330070564047
3102    84.80043739748496
